# cifar 10 loader

In [1]:
import os
import numpy as np
import tensorflow as tf
import pickle

In [2]:
# #########################
# cifar10 data file loader
# #########################
def _unpickle(file):
    with open(file, 'rb') as fo:
        item_dict = pickle.load(fo, encoding='bytes')
    return item_dict


def _load_n_preprocess(file_name):
    batch_loaded = _unpickle(file_name)
    labels = batch_loaded[b'labels']
    images = batch_loaded[b'data']

    # change type & reshape
    labels = np.asarray(labels, dtype=np.int32)

    images = np.reshape(images, newshape=(-1, 3, 32, 32))
    images = np.transpose(images, axes=(0, 2, 3, 1))
    return images, labels


def load_cifar10(cifar10_data_path):
    # download from: https://www.cs.toronto.edu/~kriz/cifar.html
    # place where data batches resides
    dataset_path = os.path.join(cifar10_data_path, 'cifar-10-batches-py')

    # parse training batch files and concat
    training_batch_images = None
    training_batch_labels = None
    for ii in range(1, 6):
        file_name = os.path.join(dataset_path, 'data_batch_{:d}'.format(ii))
        images, labels = _load_n_preprocess(file_name)

        if training_batch_images is None:
            training_batch_images = images
            training_batch_labels = labels
        else:
            training_batch_images = np.concatenate((training_batch_images, images), axis=0)
            training_batch_labels = np.concatenate((training_batch_labels, labels), axis=0)

    # parse test batch file
    file_name = os.path.join(dataset_path, 'test_batch')
    test_batch_images, test_batch_labels = _load_n_preprocess(file_name)
    return training_batch_images, training_batch_labels, test_batch_images, test_batch_labels

In [3]:
# ###############################
# Tensorflow dataset api helpers
# ###############################
IMAGE_SIZE = 32


def make_generator(images, labels):
    def _generator():
        for image, label in zip(images, labels):
            yield image, label
    return _generator


def parse_fn(images, labels):
    # convert to float32 and rescale images to -1.0 ~ 1.0
    images = tf.image.convert_image_dtype(images, dtype=tf.float32)
    images = tf.multiply(tf.subtract(images, 0.5), 2.0)
    images.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    return images, labels


def preprocess_fn(image, label, is_training):
    if is_training:
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
        image = tf.image.random_flip_left_right(image)
        image = tf.clip_by_value(image, -1.0, 1.0)

    return image, label


def graph_input_fn(cifar10_images, cifar10_labels, batch_size, is_training):
    dataset = tf.data.Dataset.from_generator(make_generator(cifar10_images, cifar10_labels), (tf.uint8, tf.int32))
    if is_training:
        dataset = dataset.shuffle(1000)
    dataset = dataset.map(parse_fn)
    dataset = dataset.map(lambda image, label: preprocess_fn(image, label, is_training))
    dataset = dataset.prefetch(batch_size)
    dataset = dataset.batch(batch_size)

    iterator = dataset.make_one_shot_iterator()
    images, labels = iterator.get_next()

    features = {
        'x': images,
    }
    return features, labels

# define network

In [4]:
# bottleneck residual unit with 3 sub layers with preactivation
def bottleneck_residual_block_v2(x, in_filter, out_filter, stage, block, is_training, strides, scale=0.0):
    l2_regularizer = tf.contrib.layers.l2_regularizer(scale=scale)
    conv_name = 'res' + str(stage) + block + '_branch'
    bn_name = 'bn' + str(stage) + block + '_branch'
    bottleneck_filter = out_filter // 4

    # conv 1x1
    x = tf.layers.batch_normalization(x, training=is_training, name=bn_name + '2a')
    x = tf.nn.relu(x)
    original_x = x
    x = tf.layers.conv2d(x, bottleneck_filter, 1, strides, padding='same', use_bias=False,
                         kernel_regularizer=l2_regularizer, name=conv_name + '2a')

    # conv 3x3
    x = tf.layers.batch_normalization(x, training=is_training, name=bn_name + '2b')
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, bottleneck_filter, 3, 1, padding='same', use_bias=False,
                         kernel_regularizer=l2_regularizer, name=conv_name + '2b')

    # conv 1x1
    x = tf.layers.batch_normalization(x, training=is_training, name=bn_name + '2c')
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, out_filter, 1, 1, padding='same', use_bias=False,
                         kernel_regularizer=l2_regularizer, name=conv_name + '2c')

    # match spatial dimension with conv2d
    if in_filter != out_filter:
        original_x = tf.layers.conv2d(original_x, out_filter, 1, strides, padding='same', use_bias=False,
                                      kernel_regularizer=l2_regularizer, name=conv_name + '1')

    x = tf.add(x, original_x)
    return x


def resnet_cifar10(images, n_classes, is_training, weight_decay):
    n_layers = 9

    # stage 1: [32, 32, 16]
    x = tf.layers.conv2d(images, filters=16, kernel_size=3, strides=1, padding='same', use_bias=False,
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay))

    # stage 2: [32, 32, 64]
    stage = 2
    block = chr(ord('a'))
    x = bottleneck_residual_block_v2(x, 16, 64, stage, block, is_training, strides=1, scale=weight_decay)
    for ii in range(1, n_layers):
        block = chr(ord(block) + 1)
        x = bottleneck_residual_block_v2(x, 64, 64, stage, block, is_training, strides=1, scale=weight_decay)

    # stage 3: [16, 16, 128]
    stage = 3
    block = chr(ord('a'))
    x = bottleneck_residual_block_v2(x, 64, 128, stage, block, is_training, strides=2, scale=weight_decay)
    for ii in range(1, n_layers):
        block = chr(ord(block) + 1)
        x = bottleneck_residual_block_v2(x, 128, 128, stage, block, is_training, strides=1, scale=weight_decay)

    # stage 4: [8, 8, 256]
    stage = 4
    block = chr(ord('a'))
    x = bottleneck_residual_block_v2(x, 128, 256, stage, block, is_training, strides=2, scale=weight_decay)
    for ii in range(1, n_layers):
        block = chr(ord(block) + 1)
        x = bottleneck_residual_block_v2(x, 256, 256, stage, block, is_training, strides=1, scale=weight_decay)

    # global average pooling
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)
    x = tf.reduce_mean(x, axis=[1, 2])

    # fully connected layer
    logits = tf.layers.dense(x, units=n_classes, kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    return logits

# define model_fn()

In [5]:
def model_fn(features, labels, mode, params):
    # parse parameters
    learning_rate = params['learning_rate']
    weight_decay = params['weight_decay']
    momentum = params['momentum']
    n_classes = params['n_classes']
    input_size = params['input_size']

    images = tf.reshape(features['x'], shape=[-1, *input_size])

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    # build networks
    logits = resnet_cifar10(images, n_classes, is_training, weight_decay)

    # ================================
    # prediction mode
    # ================================
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_id': tf.argmax(logits, axis=1),
            'probabilities': tf.nn.softmax(logits),
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # compute loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    loss += tf.losses.get_regularization_loss()

    # compute evaluation metric
    accuracy = tf.metrics.accuracy(labels=labels, predictions=tf.argmax(logits, axis=1), name='acc_op')
    tf.summary.scalar('accuracy', accuracy[1])  # during training

    # ================================
    # evaluation mode
    # ================================
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'accuracy': accuracy})

    # ================================
    # training mode
    # ================================
    assert mode == tf.estimator.ModeKeys.TRAIN

    # prepare optimizer
    if params['optimizer'] == 'adam':
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    elif params['optimizer'] == 'momentum':
        optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum)
    else:
        raise NotImplementedError

    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_ops = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_ops)

# main training function

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)

In [7]:
# grab data
cifar10_data_path = 'D:\\db'
train_x, train_y, test_x, test_y = load_cifar10(cifar10_data_path)
input_size = train_x.shape[1:]
print(input_size)

(32, 32, 3)


In [8]:
# set model path
model_name = 'ResNet_CIFAR10'
models_dir = os.path.join('./models', model_name)
print(models_dir)

./models\ResNet_CIFAR10


In [9]:
# create the Estimator
model = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=models_dir,
    config=None,
    params={
        'input_size': test_x.shape[1:],
        'n_classes': 10,
        'optimizer': 'momentum',
        'learning_rate': 0.1,
        'weight_decay': 2e-4,
        'momentum': 0.9,
    },
    warm_start_from=None
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_task_id': 0, '_eval_distribute': None, '_protocol': None, '_train_distribute': None, '_model_dir': './models\\ResNet_CIFAR10', '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016291566E48>, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_experimental_distribute': None, '_tf_random_seed': None, '_evaluation_master': '', '_device_fn': None, '_master': '', '_save_summary_steps': 100, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5}


In [10]:
# start training
epochs = 100
batch_size = 128
for e in range(0, epochs):
    model.train(input_fn=lambda: graph_input_fn(train_x, train_y, batch_size=batch_size, is_training=True))

    if e % 10 == 0:
        # evaluate
        eval_results = model.evaluate(input_fn=lambda: graph_input_fn(test_x, test_y, batch_size=100, is_training=False))
        print(eval_results)

# final evaluation
eval_results = model.evaluate(input_fn=lambda: graph_input_fn(test_x, test_y, batch_size=100, is_training=False))
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 3.1705375, step = 0
INFO:tensorflow:global_step/sec: 6.36865
INFO:tensorflow:loss = 2.3620603, step = 100 (15.703 sec)
INFO:tensorflow:global_step/sec: 10.7911
INFO:tensorflow:loss = 2.1439214, step = 200 (9.318 sec)
INFO:tensorflow:global_step/sec: 10.7422
INFO:tensorflow:loss = 2.0121799, step = 300 (9.259 sec)
INFO:tensorflow:Saving checkpoints for 391 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 1.9359593.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-02-05:30:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from 

INFO:tensorflow:loss = 0.91243047, step = 3328 (9.265 sec)
INFO:tensorflow:global_step/sec: 10.7817
INFO:tensorflow:loss = 0.75674367, step = 3428 (9.274 sec)
INFO:tensorflow:Saving checkpoints for 3519 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.8458469.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-3519
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3519 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.9703142, step = 3519
INFO:tensorflow:global_step/sec: 6.54783
INFO:tensorflow:loss = 0.7434195, step = 3619 (15.273 sec)
INFO:tensorflow:global_step/sec: 10.7193
INFO:tensorflow:loss = 0.7758165, step = 3719 (9.329 sec)
INFO:tensorflow:global_step/sec: 10.7182
INFO:tensorflow:lo

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6647 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.57640034, step = 6647
INFO:tensorflow:global_step/sec: 6.49397
INFO:tensorflow:loss = 0.63193655, step = 6747 (15.401 sec)
INFO:tensorflow:global_step/sec: 10.7609
INFO:tensorflow:loss = 0.54689157, step = 6847 (9.351 sec)
INFO:tensorflow:global_step/sec: 10.6884
INFO:tensorflow:loss = 0.6340685, step = 6947 (9.298 sec)
INFO:tensorflow:Saving checkpoints for 7038 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.589996.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-7038
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints

INFO:tensorflow:Saving checkpoints for 10166 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.70089805.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-10166
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10166 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.5005193, step = 10166
INFO:tensorflow:global_step/sec: 6.51215
INFO:tensorflow:loss = 0.5961168, step = 10266 (15.404 sec)
INFO:tensorflow:global_step/sec: 10.6667
INFO:tensorflow:loss = 0.47785136, step = 10366 (9.329 sec)
INFO:tensorflow:global_step/sec: 10.7216
INFO:tensorflow:loss = 0.64722824, step = 10466 (9.326 sec)
INFO:tensorflow:Saving checkpoints for 10557 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss fo

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13294 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.46123487, step = 13294
INFO:tensorflow:global_step/sec: 6.5237
INFO:tensorflow:loss = 0.6031892, step = 13394 (15.331 sec)
INFO:tensorflow:global_step/sec: 11.001
INFO:tensorflow:loss = 0.61397225, step = 13494 (9.091 sec)
INFO:tensorflow:global_step/sec: 10.8314
INFO:tensorflow:loss = 0.5483527, step = 13594 (9.231 sec)
INFO:tensorflow:Saving checkpoints for 13685 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.55908537.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-13685
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13685 into ./models\ResNet_C

...

INFO:tensorflow:global_step/sec: 6.38678
INFO:tensorflow:loss = 0.54813576, step = 23560 (15.658 sec)
INFO:tensorflow:global_step/sec: 9.64482
INFO:tensorflow:loss = 0.5689316, step = 23660 (10.370 sec)
INFO:tensorflow:global_step/sec: 8.65688
INFO:tensorflow:loss = 0.5254726, step = 23760 (11.553 sec)
INFO:tensorflow:Saving checkpoints for 23851 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.5704676.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-02-06:43:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-23851
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-02-06:43:59
INFO:tensorflow:Saving dict for global step 23851: accuracy = 0.8169, global_step = 23851, loss = 1.0139526
INFO:tensorflow:Saving 'checkpoint_path' summary for global 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-26979
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 26979 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.54613566, step = 26979
INFO:tensorflow:global_step/sec: 6.12524
INFO:tensorflow:loss = 0.52701306, step = 27079 (16.327 sec)
INFO:tensorflow:global_step/sec: 10.9504
INFO:tensorflow:loss = 0.53468835, step = 27179 (9.133 sec)
INFO:tensorflow:global_step/sec: 10.8655
INFO:tensorflow:loss = 0.5601512, step = 27279 (9.203 sec)
INFO:tensorflow:Saving checkpoints for 27370 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.4854627.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSav

INFO:tensorflow:loss = 0.47783822, step = 30107
INFO:tensorflow:global_step/sec: 6.50199
INFO:tensorflow:loss = 0.6135459, step = 30207 (15.381 sec)
INFO:tensorflow:global_step/sec: 10.8144
INFO:tensorflow:loss = 0.6088844, step = 30307 (9.248 sec)
INFO:tensorflow:global_step/sec: 10.8156
INFO:tensorflow:loss = 0.5647459, step = 30407 (9.245 sec)
INFO:tensorflow:Saving checkpoints for 30498 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.6336391.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-30498
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 30498 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.48289216, step = 30498
INFO:tensorflow:global_step/sec: 6.55441
INFO:tensorflow:loss =

INFO:tensorflow:Loss for final step: 0.55739903.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-33626
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 33626 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.5163117, step = 33626
INFO:tensorflow:global_step/sec: 6.66849
INFO:tensorflow:loss = 0.49694383, step = 33726 (14.998 sec)
INFO:tensorflow:global_step/sec: 11.0878
INFO:tensorflow:loss = 0.64437056, step = 33826 (9.020 sec)
INFO:tensorflow:global_step/sec: 11.0865
INFO:tensorflow:loss = 0.53546774, step = 33926 (9.019 sec)
INFO:tensorflow:Saving checkpoints for 34017 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.5992695.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calli

INFO:tensorflow:loss = 0.51589644, step = 36754
INFO:tensorflow:global_step/sec: 6.49953
INFO:tensorflow:loss = 0.623862, step = 36854 (15.389 sec)
INFO:tensorflow:global_step/sec: 10.9573
INFO:tensorflow:loss = 0.49975958, step = 36954 (9.125 sec)
INFO:tensorflow:global_step/sec: 10.9724
INFO:tensorflow:loss = 0.59401476, step = 37054 (9.114 sec)
INFO:tensorflow:Saving checkpoints for 37145 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:Loss for final step: 0.6861534.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models\ResNet_CIFAR10\model.ckpt-37145
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 37145 into ./models\ResNet_CIFAR10\model.ckpt.
INFO:tensorflow:loss = 0.6075902, step = 37145
INFO:tensorflow:global_step/sec: 6.63081
INFO:tensorflow:loss =